# 数据集生成
我们分别生成 4， 8， 12 量子比特下不同参数哈密顿量的基态，并将参数和分类结果保存。
## 4 比特情况

In [1]:
from mindquantum import *
import mindspore as ms
import numpy as np
from mindspore.nn import Adam, TrainOneStepCell
ms.set_seed(1)
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

def ham_layer(qubit_num=4, prefix='0'):
    circ_ = Circuit([ZZ(f'{0}').on([i, i+1]) for i in range(qubit_num-1)])
    circ_ += ZZ(f'{0}').on([qubit_num-1, 0])
    circ_ += Circuit([RX(f'{1}').on(i) for i in range(qubit_num)])
    circ_ += BarrierGate()
    circ_ = add_prefix(circ_, prefix)
    return circ_

def h_layer(qubit_num):
    circ_ = Circuit([H.on(i) for i in range(qubit_num)])
    circ_ += BarrierGate()
    return circ_

qubit_num = 4

encoder = h_layer(qubit_num)
encoder += ham_layer(qubit_num, prefix='0')
encoder += ham_layer(qubit_num, prefix='1')
encoder.as_ansatz()

sim = Simulator('mqvector', encoder.n_qubits)
params = np.arange(0.0, 2.0, 0.004)
train_x_set = []
train_y_set = []

k = 0
for param in params:
    if k % 10 == 0:
        print(f'进度为：{k+1}/{len(params)}')
    k += 1
    op = [QubitOperator(f'Z{i} Z{i+1}') for i in range(qubit_num-1)]
    op += [QubitOperator(f'Z{qubit_num-1} Z{0}')]
    op += [QubitOperator(f'X{i}', param) for i in range(qubit_num)]
    ham = Hamiltonian(sum(op))

    grad_ops = sim.get_expectation_with_grad(ham, encoder)
    qnet = MQAnsatzOnlyLayer(grad_ops)
    opti = Adam(qnet.trainable_params(), learning_rate=0.1)
    net = TrainOneStepCell(qnet, opti)

    res_0 = 0
    for i in range(1000):
        res = net()
        if abs(res_0 - res) <= 1e-10:
            break
        res_0 = res
    train_x_set.append(qnet.weight.asnumpy())
    train_y_set.append(param>1.)

np.save('./src/4_qubit_train_x_set.npy', np.array(train_x_set))
np.save('./src/4_qubit_train_y_set.npy', np.array(train_y_set))
print('数据收集完成啦！')

a = np.load('./src/4_qubit_train_x_set.npy', allow_pickle=True)
b = np.load('./src/4_qubit_train_y_set.npy', allow_pickle=True)
print('检查一下：')
print(a[0],len(a))
print(b[0],len(b))

[WARNING] ME(120038:139683481391936,MainProcess):2023-10-17-23:44:12.154.644 [mindspore/run_check/_check_version.py:102] MindSpore version 2.1.0 and cuda version 11.2.72 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.
/tmp/ipykernel_120038/1668479064.py:9: DeprecationWarning: ZZ gate is deprecated, please use Rzz
  circ_ = Circuit([ZZ(f'{0}').on([i, i+1]) for i in range(qubit_num-1)])
/tmp/ipykernel_120038/1668479064.py:10: DeprecationWarning: ZZ gate is deprecated, please use Rzz
  circ_ += ZZ(f'{0}').on([qubit_num-1, 0])


进度为：1/500
进度为：11/500
进度为：21/500
进度为：31/500
进度为：41/500
进度为：51/500
进度为：61/500
进度为：71/500
进度为：81/500
进度为：91/500
进度为：101/500
进度为：111/500
进度为：121/500
进度为：131/500
进度为：141/500
进度为：151/500
进度为：161/500
进度为：171/500
进度为：181/500
进度为：191/500
进度为：201/500
进度为：211/500
进度为：221/500
进度为：231/500
进度为：241/500
进度为：251/500
进度为：261/500
进度为：271/500
进度为：281/500
进度为：291/500
进度为：301/500
进度为：311/500
进度为：321/500
进度为：331/500
进度为：341/500
进度为：351/500
进度为：361/500
进度为：371/500
进度为：381/500
进度为：391/500
进度为：401/500
进度为：411/500
进度为：421/500
进度为：431/500
进度为：441/500
进度为：451/500
进度为：461/500
进度为：471/500
进度为：481/500
进度为：491/500
数据收集完成啦！
检查一下：
[ 0.45218468 -1.1179522   0.559174   -0.90465736] 500
False 500


# 8 比特情况

In [113]:
from mindquantum import *
import mindspore as ms
import numpy as np
from mindspore.nn import Adam, TrainOneStepCell
ms.set_seed(1) 
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

def ham_layer(qubit_num=4, prefix='0'):
    circ_ = Circuit([ZZ(f'{0}').on([i, i+1]) for i in range(qubit_num-1)])
    circ_ += ZZ(f'{0}').on([qubit_num-1, 0])
    circ_ += Circuit([RX(f'{1}').on(i) for i in range(qubit_num)])
    circ_ += BarrierGate()
    circ_ = add_prefix(circ_, prefix)
    return circ_

def h_layer(qubit_num):
    circ_ = Circuit([H.on(i) for i in range(qubit_num)])
    circ_ += BarrierGate()
    return circ_

qubit_num = 8

encoder = h_layer(qubit_num)
encoder += ham_layer(qubit_num, prefix='0')
encoder += ham_layer(qubit_num, prefix='1')
encoder += ham_layer(qubit_num, prefix='2')
encoder += ham_layer(qubit_num, prefix='3')
encoder.as_ansatz()

sim = Simulator('mqvector', encoder.n_qubits)
params = np.arange(0.0, 2.0, 0.004)
train_x_set = []
train_y_set = []

k = 0
for param in params:
    if k % 10 == 0:
        print(f'进度为：{k+1}/{len(params)}')
    k += 1
    op = [QubitOperator(f'Z{i} Z{i+1}') for i in range(qubit_num-1)]
    op += [QubitOperator(f'Z{qubit_num-1} Z{0}')]
    op += [QubitOperator(f'X{i}', param) for i in range(qubit_num)]
    ham = Hamiltonian(sum(op))

    grad_ops = sim.get_expectation_with_grad(ham, encoder)
    qnet = MQAnsatzOnlyLayer(grad_ops)
    opti = Adam(qnet.trainable_params(), learning_rate=0.1)  
    net = TrainOneStepCell(qnet, opti)

    res_0 = 0
    for i in range(2000):
        res = net()
        if abs(res_0 - res) <= 1e-10:
            break
        res_0 = res
    print(i)
    train_x_set.append(qnet.weight.asnumpy())
    train_y_set.append(param>1.)

np.save('./src/8_qubit_train_x_set.npy', np.array(train_x_set))
np.save('./src/8_qubit_train_y_set.npy', np.array(train_y_set))
print('数据收集完成啦！')

a = np.load('./src/8_qubit_train_x_set.npy', allow_pickle=True)
b = np.load('./src/8_qubit_train_y_set.npy', allow_pickle=True)
print('检查一下：')
print(a[0],len(a))
print(b[0],len(b))

进度为：1/500
124
164
155
161
167
147
178
144
160
176
进度为：11/500
173
150
162
152
136
172
161
132
149
164
进度为：21/500
157
140
139
180
160
156
145
152
135
128
进度为：31/500
158
145
158
111
116
137
138
145
116
146
进度为：41/500
179
135
145
171
131
137
132
148
170
158
进度为：51/500
119
149
131
199
138
140
142
125
144
140
进度为：61/500
139
160
137
143
133
122
144
153
151
154
进度为：71/500
152
182
138
137
136
137
148
154
137
164
进度为：81/500
230
140
144
137
158
170
156
149
157
172
进度为：91/500
148
123
137
153
116
152
165
158
166
147
进度为：101/500
159
128
155
158
132
122
147
158
162
158
进度为：111/500
136
160
161
129
122
167
162
165
165
207
进度为：121/500
131
137
138
151
141
150
148
144
166
142
进度为：131/500
144
155
149
153
162
148
159
162
162
137
进度为：141/500
151
144
151
157
142
165
153
157
157
183
进度为：151/500
162
174
160
167
176
153
160
161
140
160
进度为：161/500
154
202
156
164
188
143
161
159
149
153
进度为：171/500
148
133
154
151
159
167
130
217
163
182
进度为：181/500
186
141
168
164
154
172
168
191
192
242
进度为：191/500
111
166
154

In [1]:
from mindquantum import *
import mindspore as ms
import numpy as np
from mindspore.nn import Adam, TrainOneStepCell
ms.set_seed(1)
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

def ham_layer(qubit_num=4, prefix='0'):
    circ_ = Circuit([ZZ(f'{0}').on([i, i+1]) for i in range(qubit_num-1)])
    circ_ += ZZ(f'{0}').on([qubit_num-1, 0])
    circ_ += Circuit([RX(f'{1}').on(i) for i in range(qubit_num)])
    circ_ += BarrierGate()
    circ_ = add_prefix(circ_, prefix)
    return circ_

def h_layer(qubit_num):
    circ_ = Circuit([H.on(i) for i in range(qubit_num)])
    circ_ += BarrierGate()
    return circ_

qubit_num = 12

encoder = h_layer(qubit_num)
encoder += ham_layer(qubit_num, prefix='0')
encoder += ham_layer(qubit_num, prefix='1')
encoder += ham_layer(qubit_num, prefix='2')
encoder += ham_layer(qubit_num, prefix='3')
encoder += ham_layer(qubit_num, prefix='4')
encoder += ham_layer(qubit_num, prefix='5')
encoder += ham_layer(qubit_num, prefix='6')
encoder += ham_layer(qubit_num, prefix='7')
encoder += ham_layer(qubit_num, prefix='8')
encoder += ham_layer(qubit_num, prefix='9')
encoder.as_ansatz()

sim = Simulator('mqvector', encoder.n_qubits)
params = np.arange(0.0, 2.0, 0.004)
train_x_set = []
train_y_set = []

k = 0
for param in params:
    if k % 10 == 0:
        print(f'进度为：{k+1}/{len(params)}')
    k += 1
    op = [QubitOperator(f'Z{i} Z{i+1}') for i in range(qubit_num-1)]
    op += [QubitOperator(f'Z{qubit_num-1} Z{0}')]
    op += [QubitOperator(f'X{i}', param) for i in range(qubit_num)]
    ham = Hamiltonian(sum(op))

    grad_ops = sim.get_expectation_with_grad(ham, encoder)
    qnet = MQAnsatzOnlyLayer(grad_ops)
    opti = Adam(qnet.trainable_params(), learning_rate=0.1)
    net = TrainOneStepCell(qnet, opti)

    res_0 = 0
    for i in range(2000):
        res = net()
        if abs(res_0 - res) <= 1e-10:
            break
        res_0 = res
    print(i)
    train_x_set.append(qnet.weight.asnumpy())
    train_y_set.append(param>1.)

np.save('./src/12_qubit_train_x_set.npy', np.array(train_x_set))
np.save('./src/12_qubit_train_y_set.npy', np.array(train_y_set))
print('数据收集完成啦！')

a = np.load('./src/12_qubit_train_x_set.npy', allow_pickle=True)
b = np.load('./src/12_qubit_train_y_set.npy', allow_pickle=True)
print('检查一下：')
print(a[0],len(a))
print(b[0],len(b))

进度为：1/500
139
144
386
147
259
140
132
127
151
195
进度为：11/500
140
130
171
160
156
139
161
113
148
140
进度为：21/500
419
129
191
243
147
186
401
156
157
173
进度为：31/500
147
163
141
131
144
161
171
546
111
399
进度为：41/500
164
158
149
192
144
209
160
165
289
108
进度为：51/500
143
174
220
231
190
126
195
185
155
158
进度为：61/500
139
170
163
146
145
173
152
126
154
139
进度为：71/500
438
288
141
145
294
360
137
151
142
144
进度为：81/500
361
156
184
147
132
237
155
145
200
141
进度为：91/500
208
179
162
173
142
162
171
284
364
128
进度为：101/500
167
262
182
129
163
145
192
110
157
160
进度为：111/500
154
169
282
130
156
157
149
151
358
170
进度为：121/500
143
108
156
168
220
152
173
153
156
371
进度为：131/500
149
168
173
160
160
167
158
156
129
182
进度为：141/500
143
172
173
139
128
309
154
163
174
150
进度为：151/500
168
150
169
167
167
151
153
165
163
178
进度为：161/500
242
159
150
191
488
176
215
171
213
186
进度为：171/500
189
177
224
149
190
168
175
163
184
568
进度为：181/500
173
176
335
181
222
183
163
372
472
272
进度为：191/500
401
167
326

删除掉上面未收敛的 3 个点

In [13]:
import numpy as np
a = np.load('./src/12_qubit_train_x_set.npy', allow_pickle=True)
b = np.load('./src/12_qubit_train_y_set.npy', allow_pickle=True)

a = np.delete(a, 493,axis=0)
a = np.delete(a, 489,axis=0)
a = np.delete(a, 395,axis=0)

b = np.delete(b, 493,axis=0)
b = np.delete(b, 489,axis=0)
b = np.delete(b, 395,axis=0)

np.save('./src/12_qubit_train_x_set.npy', a)
np.save('./src/12_qubit_train_y_set.npy', b)
print('数据收集完成啦！')


数据收集完成啦！
